In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import requests
import random
from bs4 import BeautifulSoup
from csv import DictWriter

def fetch(url):
    try:
        src = requests.get(url)
        data = BeautifulSoup(src.text,'html.parser')
        return data
    except:
        return None
def fetch1(url):
    try:
        user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1',
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'
]
        html_text = requests.get(url, headers={'User-Agent': random.choice(user_agent_list)})
        soup=BeautifulSoup(html_text.text,'lxml')
        return soup
    except:
        return None

def fetch_403(url):
    try:
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
        src = requests.get(url,headers=headers)
        data = BeautifulSoup(src.text,'html.parser')
        return data
    except:
        return None
    
def extract(url):
    d = {}
    try:
        url1 = url+"fullcredits"
        data1 = fetch(url1)
        d['title'] = data1.h3.a.text
        d['director'] = []
        d['writer'] = []
        try:
            x = data1.find_all('table',class_='simpleTable simpleCreditsTable')
            directors = x[0].tbody.find_all('tr')
            y = {}
            for i in directors:
                for j in i.find_all('td',class_='name'):
                    dir ='https://www.imdb.com'+j.a['href']
                    #print(dir)
                    z = fetch_403(dir)
                    z = z.find('span',class_='sc-d462a8ef-6 gdrbUr starmeter-current-rank').text
                    if(z=='See rank'):
                        z = '10000'
                    y[j.a.text.replace('\n','')[1:]] = z
                #print(y)
                d['director'] = y
        except:
            d['director'] = []
        try:
            writers = x[1].tbody.find_all('tr')
            y = {}
            for i in writers:
                for j in i.find_all('td',class_='name'):
                    wri = 'https://www.imdb.com'+j.a['href']
                    z = fetch_403(wri)
                    z = z.find('span',class_='sc-d462a8ef-6 gdrbUr starmeter-current-rank').text
                    if(z=='See rank'):
                        z = '10000'
                    y[j.a.text.replace('\n','')[1:]] = z
                #print(y)
                d['writer'] = y
        except:
            d['writer'] = []

        d['genres']=[]
        data_genres=fetch1(url)
        genres1=data_genres.find('div',class_='ipc-chip-list__scroller')
        if genres1:
            genres=genres1.find_all('a',class_='ipc-chip ipc-chip--on-baseAlt')
            for genre in genres:
                if genre:
                    genre=genre.text
                    d['genres'].append(genre)
                    
        d['cast'] = []
        for i in data1.find('table',class_='cast_list').find_all('tr')[1:]:
            try:
                d['cast'].append(i.a.img['title'])
            except:
                continue
        


        d['storyline'] = []
        url4 = url+'plotsummary'
        try:
            data4 = fetch_403(url4)
            data4 = data4.find('div',class_='ipc-html-content-inner-div').text
            d['storyline'] = data4
        except:
            d['storyline'] = []
        url5 = url+'ratings'
        try:
            data5 = fetch_403(url5)
            d['avg_rating'] = data5.find('span',class_="sc-5931bdee-1 gVydpF").text
        except:
            d['avg_rating'] = []
        return d
    except:
        return d
    

def load(d):
    try:
        with open('/home/sasidharreddy/study/OELP/moremovies/maam.csv', 'a',newline='') as f_object:
            dictwriter_object = DictWriter(f_object,fieldnames=d.keys(),delimiter='\t')
            dictwriter_object.writerow(d)
            f_object.close()
    except:
        return

d = pd.read_json('/home/sasidharreddy/study/OELP/moremovies/test.json',orient='index')

for i in d[0].keys():
    i = i.replace('_',' ')
    print(i)
    try:
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
        url = 'https://www.imdb.com/'
        driver.get(url)
        search = driver.find_element(By.ID,'suggestion-search')
        submit = driver.find_element(By.ID,'suggestion-search-button')
        search.send_keys(i)
        submit.click()
        try:
            link = driver.find_element(By.XPATH,"//a[@class='ipc-metadata-list-summary-item__t']").get_attribute('href')
            driver.get(link)
            link = link.split('?')
            link = link[0]
            #print(link)
            x = extract(link)
            #print(x)
            link+='keywords/'
            driver.get(link)
            data = driver.find_element(By.XPATH,"//section[@class='ipc-page-section ipc-page-section--base']")
            k_h = {}
            k = data.find_elements(By.XPATH,"//a[@class='ipc-metadata-list-summary-item__t']")
            h = data.find_elements(By.XPATH,"//span[@class='ipc-voting__label__count ipc-voting__label__count--up']")
            for i in range(len(k)):
                k_h[k[i].text] = h[i].text
            x['keywords'] = k_h
            print(x)
            load(x)
            driver.quit()
        except:
            driver.quit()
    except:
        continue

The Rough House
{'title': 'The Rough House', 'director': {"Roscoe 'Fatty' Arbuckle": '10000', 'Buster Keaton': '10000'}, 'writer': {"Roscoe 'Fatty' Arbuckle": '10000', 'Buster Keaton': '10000', 'Joseph Anthony Roach': '10000'}, 'genres': ['Comedy', 'Short'], 'cast': ["Roscoe 'Fatty' Arbuckle", 'Al St. John', 'Buster Keaton', 'Alice Lake', 'Agnes Neilson', 'Glen Cavender', 'Josephine Stevens'], 'storyline': 'Roscoe, his wife and his mother-in-law run a seaside resort. Buster plays a gardener who puts out a fire started by Roscoe, then a delivery boy who fights with the cook St. John, then a cop.', 'avg_rating': '5.6', 'keywords': {'1910s': '0', '1900s': '0', 'newlywed': '0', 'mother': '0', 'mother in law': '0', 'newlywed couple': '0', 'bed': '0', 'bedroom': '0', 'falling asleep': '0', 'set fire': '0', 'blaze': '0', 'fire': '0', 'disaster': '0', 'gardener': '0', 'garden hose': '0', 'hose': '0', 'delivery': '0', 'delivery boy': '0', 'maid': '0', 'cook': '0', 'having a crush': '0', 'dough'